#TIME GAN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, TimeDistributed
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Simuliamo una serie temporale semplice
def generate_synthetic_data(n_samples=1000, seq_len=24):
    x = np.linspace(0, 50, n_samples)
    y = np.sin(x) + np.random.normal(0, 0.1, n_samples)  # Sinusoide con rumore
    sequences = [y[i: i + seq_len] for i in range(n_samples - seq_len)]
    return np.array(sequences)

# Normalizziamo i dati
scaler = MinMaxScaler()
data = generate_synthetic_data()
data = scaler.fit_transform(data.reshape(-1, 1)).reshape(data.shape)

# TimeGAN components
class TimeGAN:
    def __init__(self, seq_len, feature_dim, hidden_dim=24, batch_size=128):
        self.seq_len = seq_len
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.build_model()

    def build_model(self):
        # Generator
        self.generator = tf.keras.Sequential([
            LSTM(self.hidden_dim, return_sequences=True, input_shape=(self.seq_len, self.feature_dim)),
            TimeDistributed(Dense(self.feature_dim, activation='tanh'))
        ])
        
        # Discriminator
        self.discriminator = tf.keras.Sequential([
            LSTM(self.hidden_dim, return_sequences=False, input_shape=(self.seq_len, self.feature_dim)),
            Dense(1, activation='sigmoid')
        ])
        
        # Autoencoder (Embedding network)
        self.autoencoder = tf.keras.Sequential([
            LSTM(self.hidden_dim, return_sequences=True, input_shape=(self.seq_len, self.feature_dim)),
            LSTM(self.hidden_dim, return_sequences=True),
            TimeDistributed(Dense(self.feature_dim))
        ])
        
        # Compile
        self.generator.compile(optimizer='adam', loss='mse')
        self.discriminator.compile(optimizer='adam', loss='binary_crossentropy')
        self.autoencoder.compile(optimizer='adam', loss='mse')

    def train(self, data, epochs=1000):
        for epoch in range(epochs):
            idx = np.random.randint(0, data.shape[0], self.batch_size)
            real_samples = data[idx]
            noise = np.random.normal(0, 1, (self.batch_size, self.seq_len, self.feature_dim))
            fake_samples = self.generator.predict(noise)
            
            # Train discriminator
            d_loss_real = self.discriminator.train_on_batch(real_samples, np.ones((self.batch_size, 1)))
            d_loss_fake = self.discriminator.train_on_batch(fake_samples, np.zeros((self.batch_size, 1)))
            d_loss = 0.5 * (d_loss_real + d_loss_fake)
            
            # Train generator
            g_loss = self.generator.train_on_batch(noise, real_samples)
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}: D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}")

    def generate(self, num_samples=100):
        noise = np.random.normal(0, 1, (num_samples, self.seq_len, self.feature_dim))
        return self.generator.predict(noise)

# Training TimeGAN
seq_len = data.shape[1]
feature_dim = 1  # Un'unica feature (può essere esteso per più feature)
timegan = TimeGAN(seq_len, feature_dim)
timegan.train(data, epochs=1000)

# Generazione di nuove serie temporali
synthetic_data = timegan.generate(100)

# Visualizzazione
plt.plot(synthetic_data[0], label="Serie Temporale Generata")
plt.legend()
plt.show()
